# Regras de associação.

Cálculos de itemsets frequentes com o algoritmo Apriori utilizando o pacote ```mlxt```.



In [ ]:
%%capture
! pip install mlxtend

## Regras de associação geradas a partir de itemsets frequentes

Fonte: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

No exemplo a seguir, foi criado um ```dataset```  transacional formado por uma "lista de listas", onde cada linha corresponde a um cesto de compras de um supermercado hipotético.

Nesta base, são considerados ```itemsets``` frequentes aqueles que possuírem suporte superior a 0.6.

In [33]:

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Dataset transcional com cestos de compras

dataset = [['Leite', 'Cebola', 'Batata', 'Feijão', 'Ovos', 'Iogurte'],
           ['Arroz', 'Cebola', 'Batata', 'Feijão', 'Ovos', 'Iogurte'],
           ['Leite', 'Maçã', 'Feijão', 'Ovos'],
           ['Leite', 'Milho', 'Feijão', 'Iogurte'],
           ['Milho', 'Cebola', 'Feijão', 'Sorvete', 'Ovos']]

dataset

[['Leite', 'Cebola', 'Batata', 'Feijão', 'Ovos', 'Iogurte'],
 ['Arroz', 'Cebola', 'Batata', 'Feijão', 'Ovos', 'Iogurte'],
 ['Leite', 'Maçã', 'Feijão', 'Ovos'],
 ['Leite', 'Milho', 'Feijão', 'Iogurte'],
 ['Milho', 'Cebola', 'Feijão', 'Sorvete', 'Ovos']]

In [35]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset) # fit_transform TODO testar
te_ary

array([[False,  True,  True,  True,  True,  True, False, False,  True,
        False],
       [ True,  True,  True,  True,  True, False, False, False,  True,
        False],
       [False, False, False,  True, False,  True,  True, False,  True,
        False],
       [False, False, False,  True,  True,  True, False,  True, False,
        False],
       [False, False,  True,  True, False, False, False,  True,  True,
         True]])

In [36]:
te.columns_

['Arroz',
 'Batata',
 'Cebola',
 'Feijão',
 'Iogurte',
 'Leite',
 'Maçã',
 'Milho',
 'Ovos',
 'Sorvete']

In [37]:
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head()

,Arroz,Batata,Cebola,Feijão,Iogurte,Leite,Maçã,Milho,Ovos,Sorvete
0,False,True,True,True,True,True,False,False,True,False
1,True,True,True,True,True,False,False,False,True,False
2,False,False,False,True,False,True,True,False,True,False
3,False,False,False,True,True,True,False,True,False,False
4,False,False,True,True,False,False,False,True,True,True


In [40]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.6,(Cebola)
1,1.0,(Feijão)
2,0.6,(Iogurte)
3,0.6,(Leite)
4,0.8,(Ovos)
5,0.6,"(Cebola, Feijão)"
6,0.6,"(Cebola, Ovos)"
7,0.6,"(Feijão, Iogurte)"
8,0.6,"(Leite, Feijão)"
9,0.8,"(Feijão, Ovos)"


Alguma regra poderia ser inferida?



###Regras de associação

Gera regras de associação com confiança mínima de 0.7.

In [42]:
# association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [44]:
columns = ["antecedents", "consequents", "antecedent support", "consequent support", "support",	"confidence",	"lift"]

In [45]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7).sort_values("lift",ascending=False)[columns]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
1,(Cebola),(Ovos),0.6,0.8,0.6,1.00,1.25
7,"(Cebola, Feijão)",(Ovos),0.6,0.8,0.6,1.00,1.25
10,(Cebola),"(Feijão, Ovos)",0.6,0.8,0.6,1.00,1.25
2,(Ovos),(Cebola),0.8,0.6,0.6,0.75,1.25
9,"(Feijão, Ovos)",(Cebola),0.8,0.6,0.6,0.75,1.25
11,(Ovos),"(Cebola, Feijão)",0.8,0.6,0.6,0.75,1.25
0,(Cebola),(Feijão),0.6,1.0,0.6,1.00,1.00
3,(Iogurte),(Feijão),0.6,1.0,0.6,1.00,1.00
4,(Leite),(Feijão),0.6,1.0,0.6,1.00,1.00
5,(Feijão),(Ovos),1.0,0.8,0.8,0.80,1.00


Gera regras de associação com lift mínimo de 1.2. 

É importante lembrar que valores de lift inferiores a 1 significam que a regra não possui causalidade relevante e não aumentam o nosso poder de previsão.

In [46]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)[columns]
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(Cebola),(Ovos),0.6,0.8,0.6,1.00,1.25
1,(Ovos),(Cebola),0.8,0.6,0.6,0.75,1.25
2,"(Cebola, Feijão)",(Ovos),0.6,0.8,0.6,1.00,1.25
3,"(Feijão, Ovos)",(Cebola),0.8,0.6,0.6,0.75,1.25
4,(Cebola),"(Feijão, Ovos)",0.6,0.8,0.6,1.00,1.25
5,(Ovos),"(Cebola, Feijão)",0.8,0.6,0.6,0.75,1.25


In [ ]:
len(rules["antecedents"][0])

In [ ]:
rules["antecedents"].apply(lambda x: len(x))

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
columns.append("antecedent_len")
rules[columns]

Exibe apenas as regras com antecedentes de comprimento maior ou igual a 2 e com confiança superior a 0.75 e lift superior a 1.2.

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ][columns]

Exibe apenas as regras cujo consequente seja cebola

In [ ]:
rules[rules['antecedents'] == {'Ovos'}][columns]

In [ ]:
# TODO: vocês, variem as regras

## Análise de cesta de compras em Python

Fonte:  Chris Moffitt (2017), Introduction to Market Basket Analysis in Python, http://pbpython.com/market-basket-analysis.html


Neste exemplo é utilizada a base de dados **Online Retail** da UCI, disponível em [archive.ics.uci.edu/ml/machine-learning-databases/00352/Online Retail.xlsx](http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

In [ ]:
df.info()

### Preparação de dados


Comando ```strip()``` elimina espaços no início e fim da string.

Comando ```dropna()``` remove registros com valores faltantes (*missing values*) no campo ```InvoiceNo```.

Comando ```df[~df['InvoiceNo'].str.contains('C')]``` remove registros com ```InvoiceNo``` iniciados com a letra *'C'*, uma vez que esses campos correspondem a pedidos cancelados.





In [ ]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

df.describe()

Gera uma base de dados apenas com pedidos da França. É gerada uma tabela pivô em que cada coluna corresponde à um produto e cada linha corresponde ao somatório da quantidade comprada daquele produto em um determinado pedido.


In [ ]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket.head()

Transforma as quantidades em 0 ou 1.

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

basket_sets.head()

In [ ]:
basket_sets.describe()

### Geração de ```itemsets``` frequentes e de regras de associação.



In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
display(frequent_itemsets.head())

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print("\nAlgumas regras de associação geradas:\n")
display(rules.head())
print("\nDimensões da matriz de regras:", rules.shape)

### Exemplos de filtros sobre regras de associação

In [ ]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

In [ ]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

In [ ]:
basket['ALARM CLOCK BAKELIKE RED'].sum()

### Análise de cestas de compras da Alemanha

Esse código é semelhante ao código utilizado para gerar as regras da França. O objetivo é mostrar como que o suporte mínimo e a confiança mínima podem variar de uma base para outra. Um país pode ter um perfil de compras mais homogêneo e gerar regras com suporte maior, enquanto outro país pode gerar regras com suporte menor.

In [ ]:
basket2 = (df[df['Country'] == "Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

In [ ]:
basket.sum().describe()

In [ ]:
basket2.head()